<a href="https://colab.research.google.com/github/jaegon-kim/python_study/blob/main/src/ai_essential_250317/house_price_prediction/prediction_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Price Prediction - K fold 를 잘 못 사용한 예제
- **목표**
  - 이 워크샵은 주어진 데이터셋을 이용해 심층신경망 모델을 학습시켜 주택의 최종 판매 가격(SalePrice)을 예측하는 것이 최종 목표입니다.

- **데이터셋 정보**
  - 데이터셋은 총 79개의 설명 변수와 타겟 변수인 주택 가격(SalePrice)로 구성됩니다.
  - 설명 변수는 주택의 다양한 특성(예: 건축 연도, 면적, 위치, 방 개수 등)을 포함합니다.
  - 데이터는 판매 가격이 포함된 학습용 데이터인 `X`, `y` 와 판매 가격이 포함되지 않은 평가용 데이터인 `TEST`파일로 나뉘며, 각각 모델 학습 및 평가에 사용됩니다.
    - 평가용 데이터 `TEST`의 판매 가격(SalePrice)를 예측 후 리더보드로 제출하여 평가합니다.

- **문제 유형**
  - 이 워크샵은 회귀 문제로 연속형 변수를 예측하는 것이 목표입니다. 모델의 성능은 `Mean Absolute Error`로 측정됩니다.

## 1. 환경 설정

In [1]:
%%capture
!pip install JAEN -U

In [2]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from JAEN.utils import plot_training_results
from torch.utils.data import DataLoader, Subset 
from sklearn.model_selection import KFold


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/sdn/anaconda3/envs/python311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

## 2. 데이터 로드

In [3]:
from JAEN.datasets import load_house_price
X, y, TEST = load_house_price()
X.shape, y.shape, TEST.shape

(torch.Size([1460, 79]), torch.Size([1460, 1]), torch.Size([1459, 79]))

In [ ]:
#column_mean = torch.mean(X, dim=0)
#print("Column Mean:", column_mean, column_mean.shape)
#column_std = torch.std(X, dim=0)
#print("Column Std:", column_std)

In [4]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # 모델을 학습 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    for data, labels in train_loader: # 미니 배치 별 파라미터 업데이트 수행
        data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

        # 순전파
        outputs = model(data)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 기울기 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        # 손실 누적
        running_loss += loss.item()

    # 현재 Epoch의 평균 손실 값 계산 및 반환
    # - len(train_loader): 평균 Loss
    return running_loss / len(train_loader)

In [5]:
# 평가 함수 정의
def evaluate(model, test_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    with torch.no_grad():  # 평가 중에는 기울기 계산을 하지 않음
        for data, labels in test_loader: # 미니 배치 별 손실 계산
            data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

            # 순전파
            outputs = model(data)

            # 손실 계산
            loss = criterion(outputs, labels)

            # 손실 누적
            running_loss += loss.item()


    # 현재 Epoch의 평균 손실 값 계산 및 반환
    return running_loss / len(test_loader)

In [6]:
# DNN 모델 구성
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(79, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # 10%의 드롭아웃 적용

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        #x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
#model = DNN().to(device)
#summary(model)


In [7]:
# DataLoader 생성
from torch.utils.data import DataLoader, TensorDataset
#train_loader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True, drop_last=True)
dataset = TensorDataset(X, y)
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

In [ ]:
fold_models = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):
    print(f'== Fold {fold+1} / {k_folds} ==')

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)

    # Early Stopping을 위한 변수 설정
    best_loss = float('inf')
    best_epoch = 0
    patience = 200
    counter = 0
     
    train_losses = []
    test_losses = []

    # Fold 마다 Model 초기화
    model = DNN().to(device)

    # 손실함수 및 옵티마이저 설정
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

    # 학습 횟수 만큼 반복
    for epoch in range(2000):

        # 모델 학습(학습데이터)
        train_loss = train(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)

        # 모델 평가 (평가데이터)
        test_loss = evaluate(model, test_loader, criterion, device)
        test_losses.append(test_loss)

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1} Train Loss : {train_loss:,} Test Loss : {test_loss:,} in fold {fold}')

        # Early Stopping 조건 체크
        if test_loss < best_loss:
            best_loss = test_loss
            counter = 0
            best_epoch = epoch
            print(f'Save model Epoch {epoch+1} Train Loss : {train_loss:,} Test Loss : {test_loss:,}')
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            counter += 1

        if counter >= patience:
            print(f'Early stopping trigerred at epoch {epoch+1} Best Loss: {best_loss:,} in epoch {best_epoch}')
            break

    model.load_state_dict(torch.load('best_model.pt'))

    plot_training_results(train_losses, test_losses)
    plot_training_results(train_losses[2000:], test_losses[2000:])

    fold_models.append(test_loss)

== Fold 1 / 5 ==
Save model Epoch 1 Train Loss : 38,884,970,075.178085 Test Loss : 38,962,954,563.36842
Save model Epoch 2 Train Loss : 38,883,493,944.10959 Test Loss : 38,960,828,416.0
Save model Epoch 3 Train Loss : 38,881,581,673.20548 Test Loss : 38,958,415,548.63158
Save model Epoch 4 Train Loss : 38,878,964,188.9315 Test Loss : 38,956,293,389.47369
Save model Epoch 5 Train Loss : 38,875,434,811.61644 Test Loss : 38,952,309,167.1579
Save model Epoch 6 Train Loss : 38,871,293,755.61644 Test Loss : 38,947,997,157.052635
Save model Epoch 7 Train Loss : 38,865,801,833.20548 Test Loss : 38,941,366,056.42105
Save model Epoch 8 Train Loss : 38,860,042,071.671234 Test Loss : 38,934,002,202.947365
Save model Epoch 9 Train Loss : 38,853,205,062.136986 Test Loss : 38,927,009,037.47369
Save model Epoch 10 Train Loss : 38,845,919,091.72603 Test Loss : 38,922,017,414.73684
Save model Epoch 11 Train Loss : 38,837,543,950.0274 Test Loss : 38,911,423,865.26316
Save model Epoch 12 Train Loss : 38,8

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "김재곤"
assert username, "username 변수에 값이 설정되지 않았습니다."

# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='0317(1)',
    competition_name='House Price Prediction'
)

# 학습된 모델의 TEST 예측
for m in fold_models:
    with torch.no_grad():
        outputs = model(TEST)
    avg_outputs = torch.mean(torch.stack(outputs), dim=0)

#model.eval()
#TEST2 = TEST.to(device)
#with torch.no_grad():
#    outputs = model(TEST2)
#outputs

# comp.prediction에 TEST 예측 결과 대입
#comp.prediction = outputs
comp.prediction = avg_outputs
comp.prediction

# 제출
#comp.submit()